<a href="https://colab.research.google.com/github/KiranJoshi25/DataVizFall2020_Colab-/blob/master/States_activity_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import folium
import json

In [2]:
db = "https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
df = pd.read_csv(db)

In [3]:
def covid_stat_on(df,strDate):
    df_temp = df[df['date']==strDate][['state','cases','deaths']].groupby('state',as_index = False).sum().copy().dropna().reset_index(drop=True)
    return df_temp

In [4]:
state_geo_json = "https://raw.githubusercontent.com/KiranJoshi25/DataVizFall2020_Colab-/master/cb_2018_us_state_20m.json"
df_state_geo_json = pd.read_json(state_geo_json)

df_state_geo_json['features'].iloc[0]

{'geometry': {'coordinates': [[[[-76.046213, 38.025532999999996],
     [-76.04652999999999, 37.953586],
     [-75.98464799999999, 37.938120999999995],
     [-75.98008899999999, 38.004891],
     [-76.00733699999999, 38.036705999999995],
     [-76.046213, 38.025532999999996]]],
   [[[-79.47666199999999, 39.721078],
     [-79.482366, 39.531689],
     [-79.484372, 39.344300000000004],
     [-79.486873, 39.205960999999995],
     [-79.424413, 39.228170999999996],
     [-79.35374999999999, 39.278039],
     [-79.283723, 39.30964],
     [-79.26239211717609, 39.3262438284674],
     [-79.16649699999999, 39.400887999999995],
     [-79.091329, 39.472407],
     [-79.0674474080724, 39.4728086991282],
     [-79.035623, 39.473344],
     [-78.956751, 39.440264],
     [-78.942618, 39.479614],
     [-78.851016, 39.554044],
     [-78.77114, 39.638387],
     [-78.73904999999999, 39.609697],
     [-78.707098, 39.555856999999996],
     [-78.6550361715467, 39.544382241083696],
     [-78.590654, 39.530192],
   

In [5]:
state_abbrv={}
for i in range(len(df_state_geo_json['features'])):
    state_abbrv[df_state_geo_json['features'].iloc[i]['properties']['NAME']] = df_state_geo_json['features'].iloc[i]['properties']['STUSPS']
state_abbrv

{'Alabama': 'AL',
 'Alaska': 'AK',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'District of Columbia': 'DC',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Puerto Rico': 'PR',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin':

In [6]:
covid_stat = covid_stat_on(df,'2020-09-13')
covid_stat.head(1)

,state,cases,deaths
0,Alabama,138755,2351


In [7]:

state_abbrv.keys()

dict_keys(['Maryland', 'Iowa', 'Delaware', 'Ohio', 'Pennsylvania', 'Nebraska', 'Washington', 'Puerto Rico', 'Alabama', 'Arkansas', 'New Mexico', 'Texas', 'California', 'Kentucky', 'Georgia', 'Wisconsin', 'Oregon', 'Missouri', 'Virginia', 'Tennessee', 'Louisiana', 'New York', 'Michigan', 'Idaho', 'Florida', 'Alaska', 'Illinois', 'Montana', 'Minnesota', 'Indiana', 'Massachusetts', 'Kansas', 'Nevada', 'Vermont', 'Connecticut', 'New Jersey', 'District of Columbia', 'North Carolina', 'Utah', 'North Dakota', 'South Carolina', 'Mississippi', 'Colorado', 'South Dakota', 'Oklahoma', 'Wyoming', 'West Virginia', 'Maine', 'Hawaii', 'New Hampshire', 'Arizona', 'Rhode Island'])

In [29]:
covid_stat_translated = covid_stat[covid_stat['state'].isin(state_abbrv.keys())].copy()
covid_stat_translated['state'] = covid_stat_translated['state'].apply(lambda x:state_abbrv[x] )
covid_stat_translated.head()


,state,cases,deaths
0,AL,138755,2351
1,AK,7136,41
2,AZ,208514,5322
3,AR,70219,981
4,CA,763389,14386


In [34]:
import numpy as np
df_Magnitude = covid_stat_on(df,'2020-9-13')
df_Magnitude['cases']=np.log(df_Magnitude['cases'])

In [46]:


m = folium.Map(
            location=[48, -102], # center the map to this point
            zoom_start=4)        # starting zoom!


choropleth = folium.Choropleth(
             geo_data="https://raw.githubusercontent.com/KiranJoshi25/DataVizFall2020_Colab-/master/cb_2018_us_state_20m.json",
             data=covid_stat_translated,
             columns=['state', 'cases'],
             key_on='feature.properties.NAME',
             fill_color='RdPu', 
             fill_opacity=0.7,
              line_opacity=0.2,
             legend_name='Covid Cases'
             ).add_to(m)
m